In [6]:
import os
import random
import string
import csv
from captcha.image import ImageCaptcha

# Parâmetros
NUM_IMAGES = 5000
CAPTCHA_LENGTH = 5
VAL_PERCENT = 0.2
OUTPUT_BASE_DIR = "Generated/TextBased"

# Caracteres permitidos
CHARS = string.ascii_uppercase + string.digits

# Pastas e arquivos
splits = ["train", "val"]
split_counts = {
    "val": int(NUM_IMAGES * VAL_PERCENT),
    "train": NUM_IMAGES - int(NUM_IMAGES * VAL_PERCENT)
}

# Garante que pastas e arquivos CSV existem
csv_writers = {}
for split in splits:
    output_dir = os.path.join(OUTPUT_BASE_DIR, split)
    os.makedirs(output_dir, exist_ok=True)
    csv_path = os.path.join(OUTPUT_BASE_DIR, f"{split}_labels.csv")
    csv_file = open(csv_path, mode='w', newline='', encoding='utf-8')
    writer = csv.DictWriter(csv_file, fieldnames=["filename", "label"])
    writer.writeheader()
    csv_writers[split] = (writer, csv_file)

# Gerador de imagens
image_generator = ImageCaptcha(width=200, height=50, font_sizes=[38])

# Geração de todos os CAPTCHAs únicos
all_texts = set()
while len(all_texts) < NUM_IMAGES:
    all_texts.add(''.join(random.choices(CHARS, k=CAPTCHA_LENGTH)))
all_texts = list(all_texts)
random.shuffle(all_texts)

# Separa em treino e validação
val_texts = all_texts[:split_counts["val"]]
train_texts = all_texts[split_counts["val"]:]
split_map = {
    "val": val_texts,
    "train": train_texts
}

# Gera imagens e escreve nos respectivos diretórios
for split in splits:
    output_dir = os.path.join(OUTPUT_BASE_DIR, split)
    writer, _ = csv_writers[split]

    for text in split_map[split]:
        image = image_generator.generate_image(text)
        filename = f"{text}.png"
        image_path = os.path.join(output_dir, filename)
        image.save(image_path)

        writer.writerow({
            "filename": filename,
            "label": text
        })

# Fecha arquivos CSV
for writer, csv_file in csv_writers.values():
    csv_file.close()

print("\n✅ Dataset completo gerado:")
print(f"   • {split_counts['train']} imagens em '{OUTPUT_BASE_DIR}/train'")
print(f"   • {split_counts['val']} imagens em '{OUTPUT_BASE_DIR}/val'")
print(f"   • Labels salvos em CSVs correspondentes.")



✅ Dataset completo gerado:
   • 4000 imagens em 'Generated/TextBased/train'
   • 1000 imagens em 'Generated/TextBased/val'
   • Labels salvos em CSVs correspondentes.


In [7]:
import os
import random
import string
import numpy as np
import csv
from PIL import Image, ImageDraw, ImageFont

# Parâmetros
TOTAL_IMAGES          = 5000
WIDTH, HEIGHT         = 200, 50
FONT_PATH             = "fonts/COMICATE.ttf"
FONT_SIZE             = 42
BASE_DIR              = "Generated/TextBasedWave"
VAL_RATIO             = 0.2
SPLITS                = ["train", "val"]

# Cria diretórios
for split in SPLITS:
    os.makedirs(os.path.join(BASE_DIR, split), exist_ok=True)

# Texto aleatório
def generate_text(length=5):
    return ''.join(random.choices(string.ascii_uppercase, k=length))

# Gradiente horizontal
def create_gradient(width, height, start_color, end_color):
    gradient = Image.new('RGB', (width, height), color=0)
    for x in range(width):
        r = int(start_color[0] + (end_color[0] - start_color[0]) * x / width)
        g = int(start_color[1] + (end_color[1] - start_color[1]) * x / width)
        b = int(start_color[2] + (end_color[2] - start_color[2]) * x / width)
        for y in range(height):
            gradient.putpixel((x, y), (r, g, b))
    return gradient

# Distorção senoidal
def sinusoidal_distort(image, amplitude=10, period=60):
    arr = np.array(image)
    distorted = np.zeros_like(arr)
    for y in range(HEIGHT):
        shift = int(amplitude * np.sin(2 * np.pi * y / period))
        distorted[y] = np.roll(arr[y], shift, axis=0)
    return Image.fromarray(distorted)

# === Geração dos textos únicos ===
texts_gen = set()
while len(texts_gen) < TOTAL_IMAGES:
    texts_gen.add(generate_text())
texts_gen = list(texts_gen)
random.shuffle(texts_gen)

# Separa os textos
split_counts = {
    "val": int(TOTAL_IMAGES * VAL_RATIO),
    "train": TOTAL_IMAGES - int(TOTAL_IMAGES * VAL_RATIO)
}
split_texts = {
    "val": texts_gen[:split_counts["val"]],
    "train": texts_gen[split_counts["val"]:]
}

# Geração das imagens e CSVs
for split in SPLITS:
    output_dir = os.path.join(BASE_DIR, split)
    csv_path = os.path.join(BASE_DIR, f"{split}.csv")
    with open(csv_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['filename', 'label'])

        for text in split_texts[split]:
            # Cores aleatórias
            bg_start   = tuple(random.randint(100, 255) for _ in range(3))
            bg_end     = tuple(random.randint(100, 255) for _ in range(3))
            text_color = tuple(random.randint(0, 150) for _ in range(3))

            # Máscara de texto
            img = Image.new('RGB', (WIDTH, HEIGHT), color='white')
            draw = ImageDraw.Draw(img)
            font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
            tx = 20
            ty = (HEIGHT - FONT_SIZE) // 2
            draw.text((tx, ty), text, font=font, fill='black')
            mask = img.convert('L')

            # Cria imagem final
            gradient_bg  = create_gradient(WIDTH, HEIGHT, bg_start, bg_end)
            letter_layer = Image.new('RGB', (WIDTH, HEIGHT), text_color)
            colored_text = Image.composite(letter_layer, gradient_bg, mask)
            distorted_img = sinusoidal_distort(colored_text)

            # Salva imagem e CSV
            filename = f"{text}.png"
            distorted_img.save(os.path.join(output_dir, filename))
            writer.writerow([filename, text])

print("\n🎉 Dataset com distorção senoidal gerado com divisão 80/20 entre treino e validação.")



🎉 Dataset com distorção senoidal gerado com divisão 80/20 entre treino e validação.
